In [ ]:
!pip install rdflib

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from rdflib import Graph, Literal, RDF, URIRef, BNode
from rdflib.namespace import XSD, Namespace

In [5]:
OEO = Namespace("http://openenergy-platform.org/ontology/oeo/")
OEO_KG = Namespace("http://openenergy-platform.org/oekg/")
OBO = Namespace("http://purl.obolibrary.org/obo/") 
RDFS = Namespace("http://www.w3.org/2000/01/rdf-schema#")

In [6]:
import json
f = open("/content/drive/MyDrive/OEKG/new/place_holders.json")
PL = json.load(f)

In [7]:
g = Graph()
g.bind("OEO", OEO)
g.bind("OEO_KG", OEO_KG)
g.bind("OBO", OBO)

In [8]:
Scenario_Year_2015 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2015")
Scenario_Year_2020 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2020")
Scenario_Year_2025 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2025")
Scenario_Year_2030 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2030")
Scenario_Year_2035 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2035")
Scenario_Year_2040 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2040")
Scenario_Year_2045 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2045")
Scenario_Year_2050 = URIRef("http://openenergy-platform.org/ontology/oekg/scenario_year_2050")

scenario_years_dictionary = {
  "2015": Scenario_Year_2015,
  "2020": Scenario_Year_2020,
  "2025": Scenario_Year_2025, 
  "2030": Scenario_Year_2030,  
  "2035": Scenario_Year_2035,  
  "2040": Scenario_Year_2040,  
  "2045": Scenario_Year_2045,  
  "2050": Scenario_Year_2050,     
}

for k, v in scenario_years_dictionary.items():
  scenario_year_time_stamp = BNode()
  g.add((scenario_year_time_stamp, OEO.OEO_00140178, Literal(k, datatype=XSD.integer)))
  g.add((v, RDF.type, OEO.OEO_00020097))
  g.add((v, OEO.OEO_00140178, scenario_year_time_stamp))


In [9]:
scenario_duration_unit_year =  URIRef("http://openenergy-platform.org/ontology/oekg/scenario_duration_unit_year")
g.add((scenario_duration_unit_year, RDF.type, OEO.UO_0000036))

<Graph identifier=Nf47ce61e9ca040fc8f4ffa88198545d7 (<class 'rdflib.graph.Graph'>)>

In [11]:
for k, v in PL.items():
  study =  URIRef("http://openenergy-platform.org/ontology/oekg/" + k)
  g.add((study, RDF.type, OEO.OEO_00000364))
  
  scenario_horizon = BNode()
  g.add((scenario_horizon, RDF.type, OEO.OEO_00020098))
  g.add((study, OEO.BFO_0000051, scenario_horizon))

  scenario_horizon_duration = BNode()
  g.add((scenario_horizon_duration, RDF.type, OEO.OEO_00030035))
  if (v["scenario horizon"]["duration"]["unit"] == "year"):
    g.add((scenario_horizon_duration, OEO.OEO_00040010, scenario_duration_unit_year))
  scenario_horizon_duration_value = v["scenario horizon"]["duration"]["value"]
  g.add((scenario_horizon_duration, OEO.OEO_00140178, Literal(scenario_horizon_duration_value, datatype=XSD.integer)))
  g.add((scenario_horizon, OEO.BFO_0000051, scenario_horizon_duration))

  scenario_horizon_start_time = BNode()
  g.add((scenario_horizon, OEO.BFO_0000051, scenario_horizon_start_time))
  g.add((scenario_horizon_start_time, RDF.type, OEO.OEO_00030031))
  scenario_horizon_start_time_value = v["scenario horizon"]["duration"]["start time"]
  g.add((scenario_horizon_start_time, OEO.OEO_00140178, Literal(scenario_horizon_start_time_value, datatype=XSD.integer)))
  
  scenario_horizon_end_time = BNode()
  g.add((scenario_horizon, OEO.BFO_0000051, scenario_horizon_end_time))
  g.add((scenario_horizon_end_time, RDF.type, OEO.OEO_00030032))
  scenario_horizon_ending_time_value = v["scenario horizon"]["duration"]["ending time"]
  g.add((scenario_horizon_end_time, OEO.OEO_00140178, Literal(scenario_horizon_ending_time_value, datatype=XSD.integer)))
  
  for scenario_year in  v["scenario years"]:
    g.add((scenario_horizon, OEO.BFO_0000051, scenario_years_dictionary[scenario_year]))


  try:
    for sector in v["covers sector"]:
      covered_sector = BNode()
      g.add((covered_sector, RDF.type, URIRef("http://openenergy-platform.org/ontology/oeo/" + sector) ))
      g.add((study, OEO.OEO_00000505, covered_sector))

    for energy_sector in v["covers energy carrier"]:
      covered_energy_carrier = BNode()
      g.add((covered_energy_carrier, RDF.type, URIRef("http://openenergy-platform.org/ontology/oeo/" + energy_sector) ))
      g.add((study, OEO.OEO_00000523, covered_energy_carrier))
  except Exception as e: 
    print(e)


In [12]:
g.serialize(destination='OEKG.ttl', format='turtle')

<Graph identifier=Nf47ce61e9ca040fc8f4ffa88198545d7 (<class 'rdflib.graph.Graph'>)>

In [ ]:
for index, (sub, pred, obj) in enumerate(g):
  print(index, sub, pred, obj)

In [14]:
len(g)

151